<h1> <a href="https://gtfs.org/">GTFS: General Public Transit Feed Specification</a></h1>

Around the world, public transit agencies make data available about their services, routes, and stops via a standardized data format called <a href="https://gtfs.org/">GTFS</a> (originally developed by Google). 

It has two parts: the static component contains information that changes rarely including locations of stops, routes and schedules. A new version of this static information is typically released every few months. Some agencies also provide a real-time component based on live GPS data from their buses, trains etc to provide up to the minute data about vehicle positions and arrival predictions - typically updated every 30 seconds.

This practical exercise will be based on only the static GTFS data.

Start by downloading the current GTFS schedule data for South East Queensland from:
https://gtfsrt.api.translink.com.au/ (https://gtfsrt.api.translink.com.au/GTFS/SEQ_GTFS.zip)

You will need to upload the following files to your Jupyter account in the cloud:
- <code>calendar.txt</code>
- <code>routes.txt</code>
- <code>stops.txt</code>
- <code>stop_times.txt</code>
- <code>trips.txt</code>

# Finding our way to the CBD via public transport
Our goal is to travel from where we live to the Bribane CBD via public transport.
We don't know where the closest stop is, we don't know which route the trains or buses follow and we don't know when those buses or trains will arrive. 

Once you have <code>stops.txt</code> uploaded to your Jupyter account, open it from the Jupyter File Browser to view its contents.

In [1]:
# Start by reading stops.txt into a Pandas data frame using the read_csv method and set the stop_id column as the index

import pandas as pd
stops = pd.read_csv('stops.txt')# insert your code here

# display its contents
stops

/tmp/ipykernel_942/3433482991.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,platform_code
0,1,1.0,Herschel Street Stop 1 near North Quay,NaN,-27.467834,153.019079,1,https://translink.com.au/stop/000001/gtfs/,0,NaN,NaN
1,10,10.0,Ann Street Stop 10 at King George Square,NaN,-27.468003,153.023970,1,https://translink.com.au/stop/000010/gtfs/,0,NaN,NaN
2,100,100.0,Parliament Stop 94A Margaret St,NaN,-27.473751,153.026745,1,https://translink.com.au/stop/000100/gtfs/,0,NaN,NaN
3,1000,1000.0,Handford Rd at Songbird Way,NaN,-27.339069,153.043907,2,https://translink.com.au/stop/001000/gtfs/,0,NaN,NaN
4,10000,10000.0,Balcara Ave near Allira Cr,NaN,-27.344106,153.024982,2,https://translink.com.au/stop/010000/gtfs/,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
13021,place_pinesc,NaN,The Pines Shopping Centre,NaN,-28.134660,153.469767,NaN,NaN,1,NaN,NaN
13022,place_inttbl,NaN,Toombul Shopping Centre interchange,NaN,-27.408269,153.059963,NaN,NaN,1,NaN,NaN
13023,place_scuniv,NaN,University of the Sunshine Coast station,NaN,-26.718756,153.062004,NaN,NaN,1,NaN,NaN
13024,place_intuq,NaN,UQ Chancellors Place,NaN,-27.497970,153.011136,NaN,NaN,1,NaN,NaN


In [3]:
# There are thousands of stops across south east Queensland. 
# Our first goal is to find some stops near where we live.

# We start by determining the longitude and latitude of the property where we live.
# Open google maps https://www.google.com/maps and locate the property where you currently live.
# Put a pin on that location and make note of the longitude and latitude. 
# The longitude should be about 153 and the latitude about -27

my_longitude = 153.0128# insert your longitude
my_latitude = -27.4496 # insert your latitude

In [4]:
# Next we need to be able to measure the distance from our property to each of the stops. 
# To measure the distance between two pairs of longitude and latitude, we need to use a formula, 
# such as the haversine formula (https://en.wikipedia.org/wiki/Haversine_formula) to determine the
# distance between two points on a sphere (since the earth is not flat).
# The earth is not a perfect sphere, it's radius varies at different points, but we approximate its radius as 6371 kilometres.

import math

def haversine_distance(lat1, lon1, lat2, lon2):
      # convert decimal degrees to radians 
      lon1 = math.radians(lon1)
      lat1 = math.radians(lat1)
      lon2 = math.radians(lon2)
      lat2 = math.radians(lat2)
        
      # haversine formula 
      delta_lon = lon2 - lon1 
      delta_lat = lat2 - lat1 
      a =  math.sin(delta_lat/2)**2 +  math.cos(lat1) * math.cos(lat2) *  math.sin(delta_lon/2)**2
      c = 2 * math.asin(math.sqrt(a)) 
      r = 6371 # Radius of earth in kilometers.
      return c * r
    
# Test case Brisbane CBD to Nudgee
haversine_distance(-27.467834, 153.019079, -27.371936, 153.099357) # should be about 13 kilometres

13.285025279140651

In [5]:
# We can then use this function to compute the distance from our specified longitude and latitude, to each stop

def near(stop_row, lat, lon) :
    return haversine_distance(lat, lon, stop_row.stop_lat, stop_row.stop_lon)

stops['dist_from_home'] = stops.apply(near, lon=my_longitude, lat=my_latitude, axis=1)
stops # see the new column ...

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,platform_code,dist_from_home
0,1,1.0,Herschel Street Stop 1 near North Quay,NaN,-27.467834,153.019079,1,https://translink.com.au/stop/000001/gtfs/,0,NaN,NaN,2.120070
1,10,10.0,Ann Street Stop 10 at King George Square,NaN,-27.468003,153.023970,1,https://translink.com.au/stop/000010/gtfs/,0,NaN,NaN,2.324241
2,100,100.0,Parliament Stop 94A Margaret St,NaN,-27.473751,153.026745,1,https://translink.com.au/stop/000100/gtfs/,0,NaN,NaN,3.017418
3,1000,1000.0,Handford Rd at Songbird Way,NaN,-27.339069,153.043907,2,https://translink.com.au/stop/001000/gtfs/,0,NaN,NaN,12.668364
4,10000,10000.0,Balcara Ave near Allira Cr,NaN,-27.344106,153.024982,2,https://translink.com.au/stop/010000/gtfs/,0,NaN,NaN,11.791887
...,...,...,...,...,...,...,...,...,...,...,...,...
13021,place_pinesc,NaN,The Pines Shopping Centre,NaN,-28.134660,153.469767,NaN,NaN,1,NaN,NaN,88.448865
13022,place_inttbl,NaN,Toombul Shopping Centre interchange,NaN,-27.408269,153.059963,NaN,NaN,1,NaN,NaN,6.541250
13023,place_scuniv,NaN,University of the Sunshine Coast station,NaN,-26.718756,153.062004,NaN,NaN,1,NaN,NaN,81.412007
13024,place_intuq,NaN,UQ Chancellors Place,NaN,-27.497970,153.011136,NaN,NaN,1,NaN,NaN,5.381003


In [6]:
# We can then sort the stops by this new column using the sort_values method

nearby_stops = stops.sort_values(by = 'dist_from_home')# insert your code here
nearby_stops

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,platform_code,dist_from_home
2332,2066,2066.0,"Lestrange Tce near Victoria Park Rd, stop 20",NaN,-27.449206,153.012389,1,https://translink.com.au/stop/002066/gtfs/,0,NaN,NaN,0.059701
2287,2023,2023.0,"Kelvin Grove Rd at Moy Street, stop 14",NaN,-27.448303,153.010667,1,https://translink.com.au/stop/002023/gtfs/,0,NaN,NaN,0.255147
2288,2024,2024.0,"Kelvin Grove Rd at Moy Street, stop 14",NaN,-27.448364,153.010374,1,https://translink.com.au/stop/002024/gtfs/,0,NaN,NaN,0.276037
12735,898,898.0,"Kelvin Grove Rd at School Street, stop 13",NaN,-27.451720,153.011143,1,https://translink.com.au/stop/000898/gtfs/,0,NaN,NaN,0.286887
12720,876,876.0,"Kelvin Grove Rd near Prospect Tce, stop 13",NaN,-27.451532,153.010804,1,https://translink.com.au/stop/000876/gtfs/,0,NaN,NaN,0.291450
...,...,...,...,...,...,...,...,...,...,...,...,...
12044,600498,600498.0,"Cooran station, platform 1",NaN,-26.334098,152.822809,8,https://translink.com.au/stop/600498/gtfs/,0,place_crnsta,1,125.460943
12903,place_trvsta,NaN,Traveston station,NaN,-26.321113,152.784072,NaN,NaN,1,NaN,NaN,127.515846
12045,600499,600499.0,"Traveston station, platform 1",NaN,-26.320895,152.783804,8,https://translink.com.au/stop/600499/gtfs/,0,place_trvsta,1,127.544434
12047,600500,600500.0,"Gympie North station, platform 1",NaN,-26.159443,152.682805,8,https://translink.com.au/stop/600500/gtfs/,0,place_gymsta,1,147.149615


In [ ]:
# Let's choose the first of these stops and see which buses or trains are coming soon and where they are going to ...
our_stop_id = nearby_stops.index[0]
our_stop_id

In [ ]:
# Read stop_times.txt into a data frame using the read_csv method.
# Set the data type of the stop_id column to type string by adding parameter: dtype={'stop_id':'str'}

stop_times = # insert your code here

In [ ]:
# View just those stop_time rows that match our stop_id

# insert your code here

In [ ]:
# Not all of those trips will necessarily be coming today. 
# Transit agencies run different schedules on different days of the week, especially for weekends and public holidays.
# To learn about these service schedules we need to load the calendar.txt file into a data frame.
# Set the service_id column as the index and parse the two date columns as dates

services = # insert your code here
services

In [ ]:
# Start by viewing only those services that run on this day of the week.
# So, for example, if today is a Thurdsday, then we require services.thursday == 1

# insert your code here

In [ ]:
# We also need to ensure that today falls within the start_date and end_date period of that service.
# For that we need to know today's date ...
import pytz
timezone = pytz.timezone('Australia/Brisbane')
today = pandas.Timestamp.now(tz=timezone).tz_localize(None)

In [ ]:
# Find the list of service_ids for services that run today and are within the service start and end dates

todays_services = # insert your code here
todays_services

In [ ]:
# Next we need to learn which trips occur on those services, so we need to load trips.txt into a Pandas data frame.
# Set the trip_id column as the index.

trips = # insert your code here
trips

In [ ]:
# To test if a trip is part of a service, we can use the isin method
# trips.service_id.isin(todays_services)

# Find the list of trip_ids for those trips
todays_trips = # insert your code here

todays_trips

In [ ]:
# We can then use this list of trip ids to find stop times matching these trip ids.
# stop_times.trip_id.isin(todays_trips)

# Find all stop times that stop at our stop today.
# insert your code here

In [ ]:
# We aren't interested in trying to catch any trains or buses that have already departed, 
# so view only those stop times that have an arrival_time after the time now.

time_now = today.strftime('%H:%M:%S')

arriving_soon = # insert your code here
arriving_soon

In [ ]:
# That's great, but we don't know where any of these trains or buses are going to ...
# So, we start by joining this stop_time data with the trips data frame
stops_with_trips = arriving_soon.join(trips, on='trip_id')
stops_with_trips

In [ ]:
# We now have a trip_headsign column, which may help us determine where the bus or train is going
# We also now have a route_id, but it's not particularly meaningful.
# To get information about the route we need to join our stop_time and trip data with the route.txt data.

In [ ]:
# Read routes.txt into a Pandas data frame.
# Set the route_id column as the index
routes = # insert your code here

In [ ]:
# Join our stop_time and route data frame with the routes data frame based on the 'route_id' column

# insert your code here

In [ ]:
# Filter the output so that we only see the trip_id, arrival_time, route_short_name, route_long_name and trip_headsign columns

summary = # insert your code here
summary

In [ ]:
# Lets select one of those trips to explore precisely where it goes ...
our_trip_id = summary.iloc[0,0]

In [ ]:
# Find all stop_times for our trip_id (do not restrict to our stop_id)

my_stops = # insert your code here
my_stops

In [ ]:
# Unfortunately, these stop_ids don't mean anything to us,
# so we need to join this data with the stops data frame
# display only the arrival_time and stop_name

# insert your code here

In [ ]:
# Will this get us towards the Brisbane CBD? If not, explore some other options.